In [2]:
import pandas as pd
import numpy as np
# import re
import warnings
warnings.filterwarnings('ignore')

def ascii_table_to_df(table_str: str) -> pd.DataFrame:
    """
    Convert an ASCII/SQL-style table string into a pandas DataFrame.
    Automatically infers numeric and datetime column types.
    """
    rows = []
    for line in table_str.splitlines():
        if "|" in line:
            # Extract all cells between pipes
            cells = [cell.strip() for cell in line.strip().split("|")[1:-1]]
            rows.append(cells)

    # First row is header, rest is data
    columns, data = rows[0], rows[1:]

    # Build DataFrame
    df = pd.DataFrame(data, columns=columns)

    # Try to infer data types
    for col in df.columns:
        # Convert to numeric if possible
        df[col] = pd.to_numeric(df[col], errors="ignore")
        # Convert to datetime if possible
        if df[col].dtype == "object":
            try:
                df[col] = pd.to_datetime(df[col], errors="raise")
            except Exception:
                pass

    return df

In [ ]:
"""
Join tables using pd.merge(df1, df2, on='col', how='inner/left/right')
"""
# Create df_customers
data_customers = {'CustomerID': [1, 2, 3, 4],
                  'Name': ['Alice', 'Bob', 'Charlie', 'David'],
                  'City': ['New York', 'Los Angeles', 'Chicago', 'Houston']}
df_customers = pd.DataFrame(data_customers)
print('df_customers:\n', df_customers)

data_orders = {'OrderID': [101, 102, 103, 104, 105],
               'CustomerID': [1, 3, 2, 1, 5],
               'Product': ['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Speaker']}
df_orders = pd.DataFrame(data_orders)
print('df_orders:\n', df_orders)

# Inner join on 'CustomerID'
# This join returns only the rows where the join key (CustomerID) exists in both DataFrames.
inner_join_df = pd.merge(df_customers, df_orders, on='CustomerID', how='inner')
print('inner_join_df:\n', inner_join_df)

# Left join on 'CustomerID'
# This join returns all rows from the left DataFrame (df_customers) and the matching rows from the right DataFrame (df_orders). If no match is found in the right DataFrame, NaN values are used for the right DataFrame's columns.
left_join_df = pd.merge(df_customers, df_orders, on='CustomerID', how='left')
print('left_join_df:\n', left_join_df)

# Right join on 'CustomerID'
# This join returns all rows from the right DataFrame (df_orders) and the matching rows from the left DataFrame (df_customers). If no match is found in the left DataFrame, NaN values are used for the left DataFrame's columns.
right_join_df = pd.merge(df_customers, df_orders, on='CustomerID', how='right')
print('right_join_df:\n', right_join_df)

df_customers:
    CustomerID     Name         City
0           1    Alice     New York
1           2      Bob  Los Angeles
2           3  Charlie      Chicago
3           4    David      Houston
df_orders:
    OrderID  CustomerID   Product
0      101           1    Laptop
1      102           3     Mouse
2      103           2  Keyboard
3      104           1   Monitor
4      105           5   Speaker
inner_join_df:
    CustomerID     Name         City  OrderID   Product
0           1    Alice     New York      101    Laptop
1           1    Alice     New York      104   Monitor
2           2      Bob  Los Angeles      103  Keyboard
3           3  Charlie      Chicago      102     Mouse
left_join_df:
    CustomerID     Name         City  OrderID   Product
0           1    Alice     New York    101.0    Laptop
1           1    Alice     New York    104.0   Monitor
2           2      Bob  Los Angeles    103.0  Keyboard
3           3  Charlie      Chicago    102.0     Mouse
4           4 

In [ ]:
pd.pivot_table(
  df, 
  index='department',
  columns = 'region',
  values='sales',
  aggfunc = 'sum',
  fill_value=0
)

df_long = pd.melt(
    df_wide,
    id_vars=['Product'],
    var_name='Month',
    value_name='Sales'
)


In [88]:
data = {
    'Product': ['A', 'B', 'C'],
    'January': [100, 150, 200],
    'February': [120, 160, 210],
    'March': [130, 170, 220]
}
df_wide = pd.DataFrame(data)
df_long = pd.melt(
    df_wide,
    id_vars=['Product'],
    var_name='Month',
    value_name='Sales'
)
df_long


,Product,Month,Sales
0,A,January,100
1,B,January,150
2,C,January,200
3,A,February,120
4,B,February,160
5,C,February,210
6,A,March,130
7,B,March,170
8,C,March,220


Grouped operations family

groupby().agg()
    Reduces each group to summary stats.
    Tested with: multiple aggregations, custom functions, handling multiple columns.

groupby().transform()
    Aligns results back to original rows (we already covered this).
    Tested with: filling missing values, z-scores, per-group normalization.

groupby().apply()
    Flexible, but can be slower.
    Tested with: returning custom structures or row-wise computations.

groupby().filter()
    Keeps/discards whole groups based on a condition.
    Example: keep only groups with more than 3 rows.

Row/column wise transforms

apply() (DataFrame/Series)
    Tested with: row-wise or column-wise custom functions.
    Difference from transform: apply doesn’t guarantee same shape.

map() (Series only)
    Element-wise transformation.
    Example: df['col'].map(str.lower) or mapping dicts.

applymap() (DataFrame only)
    Element-wise transformation for entire DataFrame.
    Rare but sometimes tested for cleaning data.

In [110]:
data = {'Category': ['A', 'B', 'A', 'C', 'B', 'A', 'C', 'B', 'D'],
        'Value': [1, 2, 3, 4, 5, 6, 7, 8, 9]}
df = pd.DataFrame(data)
df

,Category,Value
0,A,1
1,B,2
2,A,3
3,C,4
4,B,5
5,A,6
6,C,7
7,B,8
8,D,9


In [111]:
df['grouped_cumsum'] = df.groupby('Category')['Value'].cumsum()
df = df.sort_values('Category')

In [112]:
df

,Category,Value,grouped_cumsum
0,A,1,1
2,A,3,4
5,A,6,10
1,B,2,2
4,B,5,7
7,B,8,15
3,C,4,4
6,C,7,11
8,D,9,9


In [120]:
df.sort_index(axis=1)

,Category,Value,grouped_cumsum
0,A,1,1
2,A,3,4
5,A,6,10
1,B,2,2
4,B,5,7
7,B,8,15
3,C,4,4
6,C,7,11
8,D,9,9


In [116]:
df

,Category,Value,grouped_cumsum
0,A,1,1
2,A,3,4
5,A,6,10
1,B,2,2
4,B,5,7
7,B,8,15
3,C,4,4
6,C,7,11
8,D,9,9


In [81]:
grouped = df.groupby('Category')
for name, group in grouped:
  print(name)
  print(group)

A
  Category  Value
0        A      1
2        A      3
5        A      6
B
  Category  Value
1        B      2
4        B      5
7        B      8
C
  Category  Value
3        C      4
6        C      7
D
  Category  Value
8        D      9


In [87]:
df_filtered = df.groupby('Category').filter(lambda x: len(x)>=3)
df_filtered 

,Category,Value
0,A,1
1,B,2
2,A,3
4,B,5
5,A,6
7,B,8


In [ ]:
print("Original DataFrame:")
print(df)

# Filter groups with less than 3 rows
filtered_df = df.groupby('Category').filter(lambda x: len(x) >= 3)

In [58]:
df = pd.DataFrame({
    'team': ['A', 'A', 'A', 'B', 'B', 'C'],
    'score': [10, 12, 14, 20, 22, 30]
})

df['zscore'] = df.groupby('team')['score'].transform(
    lambda x: (x - x.mean()) / x.std()
)
df

,team,score,zscore
0,A,10,-1.000000
1,A,12,0.000000
2,A,14,1.000000
3,B,20,-0.707107
4,B,22,0.707107
5,C,30,NaN


In [71]:
df.groupby('team')['score'].transform(
    # lambda x: (x - x.mean()) / x.std()
    func='mean'
).reset_index(name='zscore')

,index,zscore
0,0,12.0
1,1,12.0
2,2,12.0
3,3,21.0
4,4,21.0
5,5,30.0


In [76]:
df['rank_in_team'] = df.groupby('team')['score'].transform(
    # lambda x: x.rank(ascending=False)
    lambda x: (x-x.min())/(x.max()-x.min())
)
df

,team,score,zscore,rank_in_team
0,A,10,-1.000000,0.0
1,A,12,0.000000,0.5
2,A,14,1.000000,1.0
3,B,20,-0.707107,0.0
4,B,22,0.707107,1.0
5,C,30,NaN,NaN


# some basics

## dataset conversion 

In [129]:
# convert dict to dataframe
data_dict = {
  'Category': ['A', 'B', 'A', 'C', 'B'],
  'Value': [10, 15, 20, 25, 30]
}

df = pd.DataFrame(data_dict)
df

,Category,Value
0,A,10
1,B,15
2,A,20
3,C,25
4,B,30


In [134]:
df['Value'].size-df['Value'].count()

0

In [136]:
df['Category'].value_counts().sum()

5

In [61]:
# convert dataframe to dict 
df.to_dict(orient='list')

{'Category': ['A', 'B', 'A', 'C', 'B'], 'Value': [10, 15, 20, 25, 30]}

## set index, reset_index

In [ ]:
"""
set_index()
"""
# Create a sample DataFrame
data = {'ID': [1, 2, 3, 4],
        'Name': ['Alice', 'Bob', 'Charlie', 'David'],
        'Age': [25, 30, 35, 40]}
df = pd.DataFrame(data)

print("Original DataFrame:")
print(df)

# Set 'ID' column as the index
df_indexed = df.set_index('ID')

print("\nDataFrame with 'ID' as index:")
print(df_indexed)

Original DataFrame:
   ID     Name  Age
0   1    Alice   25
1   2      Bob   30
2   3  Charlie   35
3   4    David   40

DataFrame with 'ID' as index:
       Name  Age
ID              
1     Alice   25
2       Bob   30
3   Charlie   35
4     David   40


## remove columns, nulls

In [ ]:
df.dropna(subset=['price'], inplace=True) # means drop rows of price column that are null, do it in place without creating a new copy

In [ ]:
df.drop('B', axis=1) # means "remove the column named 'B' from the DataFrame df

In [ ]:
# use astype to convert types
data = {'col_float': [1.0, 2.0, 3.0, 4.0, None]}
df = pd.DataFrame(data)
df['col_float'].astype('Int64')

0       1
1       2
2       3
3       4
4    <NA>
Name: col_float, dtype: Int64

## astype: change data types

In [53]:
df = pd.DataFrame({
    'Name': ['Alice', 'Bob', 'Charlie'],
    'Age_str': ['25', '30', '35'],
    'Salary_str': ['50000', '60000', '75000']
})
df['Age_str'].astype(int)

0    25
1    30
2    35
Name: Age_str, dtype: int64

## stack, unstack

In [ ]:
# stack() and unstack() methods are used for reshaping DataFrames, particularly when dealing with MultiIndex objects. They allow you to transform data between "wide" and "long" formats.

# The stack() method transforms a DataFrame from a "wide" format to a "long" format by pivoting the column labels into a new innermost level of the row index (MultiIndex).

# Create a DataFrame with a MultiIndex in columns
columns = pd.MultiIndex.from_tuples([('A', 'one'), ('A', 'two'), ('B', 'one'), ('B', 'two')])
df = pd.DataFrame(np.random.randn(3, 4), index=['X', 'Y', 'Z'], columns=columns)
print("Original DataFrame:")
print(df)

# Stack the columns
df_stacked = df.stack()
print("\nStacked DataFrame:")
print(df_stacked)

Original DataFrame:
          A                   B          
        one       two       one       two
X -0.767164  1.578430  0.466429 -1.467368
Y  0.877599 -0.515795  0.918338  0.739397
Z  0.218207  1.392626  0.283547 -0.648213

Stacked DataFrame:
              A         B
X one -0.767164  0.466429
  two  1.578430 -1.467368
Y one  0.877599  0.918338
  two -0.515795  0.739397
Z one  0.218207  0.283547
  two  1.392626 -0.648213


In [126]:
# The unstack() method performs the inverse operation of stack(). It pivots a level of the hierarchical row index (MultiIndex) into a new innermost level of the column labels.

# Create a Series with a MultiIndex
index = pd.MultiIndex.from_tuples([('red', 'p'), ('red', 'q'), ('blue', 'p'), ('blue', 'q')])
s = pd.Series(np.arange(6.0, 10.0), index=index)
print("Original Series:")
print(s)

# Unstack the last level of the index (default)
s_unstacked_default = s.unstack()
print("\nUnstacked Series (default level):")
print(s_unstacked_default)

# Unstack a specific level (e.g., level 0)
s_unstacked_level0 = s.unstack(level=0)
print("\nUnstacked Series (level 0):")
print(s_unstacked_level0)

Original Series:
red   p    6.0
      q    7.0
blue  p    8.0
      q    9.0
dtype: float64

Unstacked Series (default level):
        p    q
blue  8.0  9.0
red   6.0  7.0

Unstacked Series (level 0):
   blue  red
p   8.0  6.0
q   9.0  7.0


# string methods

In [ ]:
""" 1683
find the IDs of invalid tweets, where the content of the tweet is greater than 15:
"""
# dataframe.loc[index, columns]
# series.str.len()

df = ascii_table_to_df("""
+----------+-----------------------------------+
| tweet_id | content                           |
+----------+-----------------------------------+
| 1        | Let us Code                       |
| 2        | More than fifteen chars are here! |
+----------+-----------------------------------+
""")

def func(df: pd.DataFrame) -> pd.DataFrame:
  return df.loc[df['content'].str.len()>15, ['tweet_id']]
func(df)

,tweet_id
1,2


In [ ]:
""" 1873
Write a solution to calculate the bonus of each employee. The bonus of an employee is 100% of their salary if the ID of the employee is an odd number and the employee's name does not start with the character 'M'. The bonus of an employee is 0 otherwise. Return the result table ordered by employee_id.
"""
# series.str.startswith(char)
# dataframe.sort_values(by=column, ascending=True)

df = ascii_table_to_df("""
+-------------+---------+--------+
| employee_id | name    | salary |
+-------------+---------+--------+
| 2           | Meir    | 3000   |
| 3           | Michael | 3800   |
| 7           | Addilyn | 7400   |
| 8           | Juan    | 6100   |
| 9           | Kannon  | 7700   |
+-------------+---------+--------+
""")

def calculate_special_bonus(df: pd.DataFrame) -> pd.DataFrame:
    # create a new column for bonus with default value 0
    df['bonus'] = 0

    # calculate bonus based on the conditions
    df.loc[(df['employee_id']%2 !=0) & (~df['name'].str.startswith('M')), 'bonus'] = df['salary']

    # select only the required columns and sort the result table by employee_id in ascending order
    result_df = df[['employee_id', 'bonus']].sort_values(by='employee_id', ascending=True)

    return result_df
calculate_special_bonus(df)


,employee_id,bonus
0,2,0
1,3,0
2,7,7400
3,8,0
4,9,7700


In [ ]:
""" 1667
Write a solution to fix the names so that only the first character is uppercase and the rest are lowercase.
Return the result table ordered by user_id.
"""
# series.capitalize(): only make the first letter uppercase
# series.title(): make each word capitalized, e.g. "john doe" → "John Doe" 
# notice, keyword 'by' in sort_values() only works for dataframe; no 'by' keyword for series

df = ascii_table_to_df("""
+---------+-------+
| user_id | name  |
+---------+-------+
| 1       | aLice |
| 2       | bOB   |
+---------+-------+
""")
def func(df: pd.DataFrame) -> pd.DataFrame:
  df['name'] = df['name'].str.capitalize()
  return df.sort_values(by='user_id', ascending=True)
func(df)

,user_id,name
0,1,Alice
1,2,Bob


In [58]:
""" 1517
Write a solution to find the users who have valid emails.
A valid e-mail has a prefix name and a domain where:
    The prefix name is a string that may contain letters (upper or lower case), digits, underscore '_', period '.', and/or dash '-'. The prefix name must start with a letter.
    The domain is '@leetcode.com'.
Return the result table in any order.
"""
# regex filtering
# series.str.match(pattern, na=False)

df = ascii_table_to_df("""
+---------+-----------+-------------------------+
| user_id | name      | mail                    |
+---------+-----------+-------------------------+
| 1       | Winston   | winston@leetcode.com    |
| 2       | Jonathan  | jonathanisgreat         |
| 3       | Annabelle | bella-@leetcode.com     |
| 4       | Sally     | sally.come@leetcode.com |
| 5       | Marwan    | quarz#2020@leetcode.com |
| 6       | David     | david69@gmail.com       |
| 7       | Shapiro   | .shapo@leetcode.com     |
+---------+-----------+-------------------------+
""")
def func(df: pd.DataFrame) -> pd.DataFrame:
  pattern = r'^[A-Za-z][A-Za-z0-9_.-]*@leetcode\.com$'
  return df.loc[df['mail'].str.match(pattern, na=False)] 
func(df)

,user_id,name,mail
0,1,Winston,winston@leetcode.com
2,3,Annabelle,bella-@leetcode.com
3,4,Sally,sally.come@leetcode.com


In [59]:
""" 1527
Write a solution to find the patient_id, patient_name, and conditions of the patients who have Type I Diabetes. Type I Diabetes always starts with DIAB1 prefix.
Return the result table in any order.
"""


df = ascii_table_to_df("""
+------------+--------------+--------------+
| patient_id | patient_name | conditions   |
+------------+--------------+--------------+
| 1          | Daniel       | YFEV COUGH   |
| 2          | Alice        |              |
| 3          | Bob          | DIAB100 MYOP |
| 4          | George       | ACNE DIAB100 |
| 5          | Alain        | DIAB201      |
+------------+--------------+--------------+
""")
def func(df: pd.DataFrame) -> pd.DataFrame:
  return df
func(df)

,patient_id,patient_name,conditions
0,1,Daniel,YFEV COUGH
1,2,Alice,
2,3,Bob,DIAB100 MYOP
3,4,George,ACNE DIAB100
4,5,Alain,DIAB201


# data manipulation

In [ ]:
""" 177
Write a solution to find the nth highest distinct salary from the Employee table. If there are less than n distinct salaries, return null.
"""


table = """
+----+--------+
| id | salary |
+----+--------+
| 1  | 100    |
| 2  | 200    |
| 3  | 300    |
+----+--------+
"""
df = ascii_table_to_df(table)
def func(df: pd.DataFrame) -> pd.DataFrame:
  return df
func(df)

,id,salary
0,1,100
1,2,200
2,3,300


In [56]:
""" 176
Write a solution to find the second highest distinct salary from the Employee table. If there is no second highest salary, return null (return None in Pandas).
"""


table = """
+----+--------+
| id | salary |
+----+--------+
| 1  | 100    |
| 2  | 200    |
| 3  | 300    |
+----+--------+
"""
df = ascii_table_to_df(table)
def func(df: pd.DataFrame) -> pd.DataFrame:
  return df
func(df)

,id,salary
0,1,100
1,2,200
2,3,300


# data aggregation


Pandas data aggregation involves summarizing or combining data within a DataFrame or Series, often after grouping it by one or more columns. This process reduces the dimensionality of the data, providing insights into trends and patterns.


Key Concepts and Methods:

    groupby(): This is the fundamental method for splitting a DataFrame into groups based on the unique values in one or more columns. It returns a DataFrameGroupBy object, which can then be aggregated.

Aggregation Functions:

After grouping, various aggregation functions can be applied to the grouped data to perform calculations like:

    sum(): Computes the sum of values in each group.
    mean(): Calculates the average of values in each group.
    median(): Determines the median of values in each group.
    min(): Finds the minimum value in each group.
    max(): Finds the maximum value in each group.
    count(): Counts the number of non-null values in each group.
    std(): Computes the standard deviation of values in each group.
    var(): Computes the variance of values in each group.
    size(): Computes the size (number of rows) of each group.
    first(), last(), nth(): Retrieve specific values from each group.


## group a numerical column into bins of finite size

In [137]:

# Sample DataFrame
data = {'value': np.random.rand(20) * 100,
        'category': np.random.choice(['A', 'B', 'C'], 20)}
df = pd.DataFrame(data)

# Define your bin edges
bins = [0, 20, 40, 60, 80, 100]
df

,value,category
0,56.607428,C
1,13.534365,A
2,87.135346,A
3,96.895081,B
4,99.726637,C
5,10.453938,A
6,64.362708,C
7,14.359092,A
8,3.715306,A
9,81.180637,A


In [138]:
df['binned_value'] = pd.cut(df['value'], bins=bins)
df

,value,category,binned_value
0,56.607428,C,"(40, 60]"
1,13.534365,A,"(0, 20]"
2,87.135346,A,"(80, 100]"
3,96.895081,B,"(80, 100]"
4,99.726637,C,"(80, 100]"
5,10.453938,A,"(0, 20]"
6,64.362708,C,"(60, 80]"
7,14.359092,A,"(0, 20]"
8,3.715306,A,"(0, 20]"
9,81.180637,A,"(80, 100]"


In [139]:
# Group by the binned values and count occurrences in each bin
grouped_data = df.groupby('binned_value').size()
print("Count of items in each bin:")
print(grouped_data)

# Group by both category and binned value, then calculate the mean of 'value'
grouped_by_category_and_bin = df.groupby(['category', 'binned_value'])['value'].mean()
print("\nMean 'value' per category and bin:")
print(grouped_by_category_and_bin)

Count of items in each bin:
binned_value
(0, 20]      8
(20, 40]     3
(40, 60]     2
(60, 80]     2
(80, 100]    5
dtype: int64

Mean 'value' per category and bin:
category  binned_value
A         (0, 20]          9.925885
          (20, 40]        31.506553
          (40, 60]              NaN
          (60, 80]        76.633549
          (80, 100]       84.157991
B         (0, 20]         10.882210
          (20, 40]              NaN
          (40, 60]              NaN
          (60, 80]              NaN
          (80, 100]       91.696546
C         (0, 20]               NaN
          (20, 40]        25.034617
          (40, 60]        56.602691
          (60, 80]        64.362708
          (80, 100]       99.726637
Name: value, dtype: float64


In [144]:
df.head()

,value,category,binned_value
0,56.607428,C,"(40, 60]"
1,13.534365,A,"(0, 20]"
2,87.135346,A,"(80, 100]"
3,96.895081,B,"(80, 100]"
4,99.726637,C,"(80, 100]"


In [ ]:
df = pd.DataFrame({
  'Category': ['A', 'B', 'A', 'C', 'B'],
  'Value': [10, 15, 20, 25, 30]
})
df

,Category,Value
0,A,10
1,B,15
2,A,20
3,C,25
4,B,30


In [64]:
# Group by 'Category' and calculate the sum of 'Value' for each group
df.groupby('Category')['Value'].sum() # this returns a Series, with Category as the index

Category
A    30
B    45
C    25
Name: Value, dtype: int64

In [65]:
df.groupby('Category')['Value'].sum().reset_index() # this return a DataFrame, with Category as a column

,Category,Value
0,A,30
1,B,45
2,C,25


aggregate() or agg():
This method provides more flexibility for applying multiple aggregation functions to one or more columns, or different functions to different columns.

## multiple aggregations

In [143]:
df.groupby('Category')['Value'].agg(['sum', 'mean']) # Apply multiple aggregations to a single column


,sum,mean
Category,,
A,30,15.0
B,45,22.5
C,25,25.0


In [144]:
# alternatively, 
df.groupby('Category').agg(
  sum=('Value', 'sum'),
  mean=('Value', 'mean')
)

,sum,mean
Category,,
A,30,15.0
B,45,22.5
C,25,25.0


In [145]:
# alternatively, 
df.groupby('Category')['Value'].agg(
  sum='sum',
  mean='mean'
)

,sum,mean
Category,,
A,30,15.0
B,45,22.5
C,25,25.0


In [66]:
# Apply different aggregations to different columns
df = pd.DataFrame({
  'Category': ['A', 'B', 'A', 'C', 'B'],
  'Value1': [10, 15, 20, 25, 30],
  'Value2': [1, 2, 3, 4, 5]
})
df

,Category,Value1,Value2
0,A,10,1
1,B,15,2
2,A,20,3
3,C,25,4
4,B,30,5


In [67]:
df.groupby('Category').agg(
    total_value1=('Value1', 'sum'),
    avg_value2=('Value2', 'mean')
)

,total_value1,avg_value2
Category,,
A,30,2.0
B,45,3.5
C,25,4.0


In [148]:
# User-defined functions can be passed to agg() for more specialized aggregation logic.

In [69]:
df = pd.DataFrame({
  'Category': ['A', 'B', 'A', 'C', 'B'],
  'Value': [10, 15, 20, 25, 30]
})
def custom_range(series):
    return series.max() - series.min()
df.groupby('Category')['Value'].agg(custom_range)

Category
A    10
B    15
C     0
Name: Value, dtype: int64

In [70]:
# alternatively
df.groupby('Category')['Value'].apply(custom_range)

Category
A    10
B    15
C     0
Name: Value, dtype: int64

##### Use .agg() → when your function is truly an aggregation (scalar result per group).

#### Use .apply() → when your function may return non-scalar results or you need more flexibility.

In [155]:
df = pd.DataFrame({
    "Category": ["A","A","B","B","C","C"],
    "Value": [5, 17, 3, 8, 10, 19]
})
df

,Category,Value
0,A,5
1,A,17
2,B,3
3,B,8
4,C,10
5,C,19


In [156]:
df.groupby("Category")["Value"].agg(
    range_min="min",
    range_max="max"
)

,range_min,range_max
Category,,
A,5,17
B,3,8
C,10,19


In [157]:
def custom_range(series):
    return series.max() - series.min()
  
df.groupby("Category")["Value"].agg(
    range_min="min",
    range_max="max",
    range = custom_range
)

,range_min,range_max,range
Category,,,
A,5,17,12
B,3,8,5
C,10,19,9


In [161]:
def custom_range(series):
  return pd.Series({
    'min': series.min(),
    'max': series.max(),
    'range': series.max() - series.min()
  })

df.groupby("Category")["Value"].apply(custom_range).reset_index()

,Category,level_1,Value
0,A,min,5
1,A,max,17
2,A,range,12
3,B,min,3
4,B,max,8
5,B,range,5
6,C,min,10
7,C,max,19
8,C,range,9


In [ ]:
""" 2356
Write a solution to calculate the number of unique subjects each teacher teaches in the university.
Return the result table in any order.
"""

df = ascii_table_to_df("""
+------------+------------+---------+
| teacher_id | subject_id | dept_id |
+------------+------------+---------+
| 1          | 2          | 3       |
| 1          | 2          | 4       |
| 1          | 3          | 3       |
| 2          | 1          | 1       |
| 2          | 2          | 1       |
| 2          | 3          | 1       |
| 2          | 4          | 1       |
+------------+------------+---------+
""")
def func(df: pd.DataFrame) -> pd.DataFrame:
  return df.groupby('teacher_id')['subject_id'].nunique().reset_index(name='cnt')
func(df)
# this will drop dept_id, because nunique() operates on subject_id not dept_id 
# subject_id and dept_id will have different sizes

,teacher_id,cnt
0,1,2
1,2,4


In [ ]:
""" 1484
Write a solution to find for each date the number of different products sold and their names.
The sold products names for each date should be sorted lexicographically.
Return the result table ordered by sell_date
"""
activities = ascii_table_to_df("""
+------------+------------+
| sell_date  | product     |
+------------+------------+
| 2020-05-30 | Headphone  |
| 2020-06-01 | Pencil     |
| 2020-06-02 | Mask       |
| 2020-05-30 | Basketball |
| 2020-06-01 | Bible      |
| 2020-06-02 | Mask       |
| 2020-05-30 | T-Shirt    |
+------------+------------+
""")
def categorize_products(activities: pd.DataFrame) -> pd.DataFrame:
  return activities.groupby('sell_date').agg(
    num_sold = ('product', 'nunique'),
    products = ('product', lambda x: ','.join(sorted(x.unique())))
  ).reset_index()
categorize_products(activities)

,sell_date,num_sold,products
0,2020-05-30,3,"Basketball,Headphone,T-Shirt"
1,2020-06-01,2,"Bible,Pencil"
2,2020-06-02,1,Mask


In [78]:
"""
For each date_id and make_name, find the number of distinct lead_id's and distinct partner_id's.
Return the result table in any order.
"""
daily_sales = ascii_table_to_df("""
+-----------+-----------+---------+------------+
| date_id   | make_name | lead_id | partner_id |
+-----------+-----------+---------+------------+
| 2020-12-8 | toyota    | 0       | 1          |
| 2020-12-8 | toyota    | 1       | 0          |
| 2020-12-8 | toyota    | 1       | 2          |
| 2020-12-7 | toyota    | 0       | 2          |
| 2020-12-7 | toyota    | 0       | 1          |
| 2020-12-8 | honda     | 1       | 2          |
| 2020-12-8 | honda     | 2       | 1          |
| 2020-12-7 | honda     | 0       | 1          |
| 2020-12-7 | honda     | 1       | 2          |
| 2020-12-7 | honda     | 2       | 1          |
+-----------+-----------+---------+------------+
""")
def daily_leads_and_partners(daily_sales: pd.DataFrame) -> pd.DataFrame:
    return daily_sales.groupby(['date_id','make_name']).agg(
      unique_leads = ('lead_id', 'nunique'),
      unique_partners = ('partner_id', 'nunique')
    ).reset_index()
daily_leads_and_partners(daily_sales)

,date_id,make_name,unique_leads,unique_partners
0,2020-12-07,honda,3,2
1,2020-12-07,toyota,1,2
2,2020-12-08,honda,2,2
3,2020-12-08,toyota,2,3


## an employees case

In [100]:
df = pd.read_csv('./employees.csv', parse_dates=['hire_date'])
# alternatively
# df = pd.read_csv('./employees.csv', )
# df['hire_date'] = pd.to_datetime(df['hire_date'])
df

,employee_id,department,manager,hire_date,salary,region,sales
0,101,Sales,Alice,2018-05-10,70000,East,150000
1,102,Sales,Alice,2020-07-12,68000,East,120000
2,103,Engineering,Bob,2019-03-05,95000,West,50000
3,104,Engineering,Bob,2021-11-20,87000,West,60000
4,105,HR,Carol,2017-01-15,60000,East,0
5,106,HR,Carol,2019-08-22,62000,West,0
6,107,Sales,Alice,2022-04-01,71000,West,80000
7,108,Engineering,Bob,2018-09-09,98000,East,70000
8,109,Sales,Alice,2019-12-17,69000,West,90000
9,110,HR,Carol,2021-06-05,64000,East,0


In [102]:
df.dtypes

employee_id             int64
department             object
manager                object
hire_date      datetime64[ns]
salary                  int64
region                 object
sales                   int64
dtype: object

In [105]:
"""
count employees by department
"""
df.groupby('department').size().reset_index(name='num_employees')

,department,num_employees
0,Engineering,3
1,HR,3
2,Sales,4


In [ ]:
"""
average salary per department
"""
df.groupby('department')['salary'].mean().reset_index(name='avg_salary')

,department,avg_salary
0,Engineering,93333.333333
1,HR,62000.000000
2,Sales,69500.000000


In [107]:
"""
total and average sales per region 
"""
df.groupby('region')['sales'].agg(
  total_sales='sum',
  avg_sales='mean'
).reset_index()

,region,total_sales,avg_sales
0,East,340000,68000.0
1,West,280000,56000.0


In [119]:
"""
earliest and latest hire data per manager
"""
df.groupby('manager')['hire_date'].agg(
  first_hire='min',
  last_hire='max'
).reset_index()

,manager,first_hire,last_hire
0,Alice,2018-05-10,2022-04-01
1,Bob,2018-09-09,2021-11-20
2,Carol,2017-01-15,2021-06-05


In [121]:
"""
distint count of employees per department and region 
"""
df.groupby(['department', 'region'])['employee_id'].nunique().reset_index(name='cnt')

,department,region,cnt
0,Engineering,East,1
1,Engineering,West,2
2,HR,East,2
3,HR,West,1
4,Sales,East,2
5,Sales,West,2


In [ ]:
"""
for each department, list the top 1 highest paid employees
"""
# .head() works row-wise, not as a group reducer
df.sort_values('salary', ascending=False).groupby('department').head(1)

,employee_id,department,manager,hire_date,salary,region,sales
7,108,Engineering,Bob,2018-09-09,98000,East,70000
6,107,Sales,Alice,2022-04-01,71000,West,80000
9,110,HR,Carol,2021-06-05,64000,East,0


In [ ]:
"""
for each manager, collect employee IDs as a comma-separated string
"""
# use apply (although using agg has similar effect, as both work as a group reducer in this case)
# employee_id is of type int, use map(str, x) to convert to str first
df.groupby('manager')['employee_id'].apply(
  lambda x: ','.join(sorted(map(str, x)))
).reset_index(name='employee_ids')

,manager,employee_ids
0,Alice,"101,102,107,109"
1,Bob,"103,104,108"
2,Carol,"105,106,110"


In [139]:
"""
for each region, what % of total company sales does it contribute
"""
region_sales = df.groupby('region')['sales'].sum().reset_index()
region_sales['sale_pct'] = round(region_sales['sales']/region_sales['sales'].sum()*100, 2)
region_sales

,region,sales,sale_pct
0,East,340000,54.84
1,West,280000,45.16


In [ ]:
"""
monthly average salary trend per department
"""
# group by month: pd.Grouper(key='hire_date', freq='ME')
df.groupby([
  'department',
  pd.Grouper(key='hire_date', freq='ME')
])['salary'].mean().reset_index()

,department,hire_date,salary
0,Engineering,2018-09-30,98000.0
1,Engineering,2019-03-31,95000.0
2,Engineering,2021-11-30,87000.0
3,HR,2017-01-31,60000.0
4,HR,2019-08-31,62000.0
5,HR,2021-06-30,64000.0
6,Sales,2018-05-31,70000.0
7,Sales,2019-12-31,69000.0
8,Sales,2020-07-31,68000.0
9,Sales,2022-04-30,71000.0


In [144]:
"""
pivot table of total sales by department and region
"""
pd.pivot_table(
  df, 
  index='department',
  columns = 'region',
  values='sales',
  aggfunc = 'sum',
  fill_value=0
)

region,East,West
department,,
Engineering,70000,110000
HR,0,0
Sales,270000,170000


In [146]:
"""
top 2 employees by sales in each region
"""
df.sort_values('sales', ascending=False).groupby('region').head(2)

,employee_id,department,manager,hire_date,salary,region,sales
0,101,Sales,Alice,2018-05-10,70000,East,150000
1,102,Sales,Alice,2020-07-12,68000,East,120000
8,109,Sales,Alice,2019-12-17,69000,West,90000
6,107,Sales,Alice,2022-04-01,71000,West,80000


In [151]:
"""
rolling average of sales per region (last 2 hires)
"""
# rolling over last 2 hires, so it needs to sort accoding to hire_date first
df.sort_values('hire_date').groupby('region')['sales'].rolling(2).mean().reset_index(name='rolling_avg_sale_last_2_hires')

,region,level_1,rolling_avg_sale_last_2_hires
0,East,4,NaN
1,East,0,75000.0
2,East,7,110000.0
3,East,1,95000.0
4,East,9,60000.0
5,West,2,NaN
6,West,5,25000.0
7,West,8,45000.0
8,West,3,75000.0
9,West,6,70000.0


In [153]:
"""
percentage of employees in each department relative to company
"""
dept_empl = df.groupby('department')['employee_id'].nunique().reset_index(name='empl_cnt')
dept_empl['dept_empl_pct'] = dept_empl['empl_cnt']/dept_empl['empl_cnt'].sum()*100
dept_empl

,department,empl_cnt,dept_empl_pct
0,Engineering,3,30.0
1,HR,3,30.0
2,Sales,4,40.0


# partition after aggregation

In [ ]:
"""

The pandas.DataFrame.transform() and pandas.Series.transform() methods in the Pandas library are used for applying a function to a DataFrame or Series, respectively, and returning a new object with the same axis shape as the original. This means that the output will have the same number of rows (and columns, for DataFrames) as the input.
Key characteristics and uses of transform():

    Preserves shape:
    The most crucial aspect of transform() is that it always returns an object with the same index and shape as the original. This is in contrast to aggregation methods (like sum(), mean(), etc.) which reduce the data to a single value per group, or apply() which can return objects of varying shapes.

    Group-level operations with row-level results:
    When used with groupby(), transform() allows you to perform calculations at the group level and then broadcast the results back to each individual row within that group. For example, you can calculate the mean of a group and then add that mean to every element in the original column for that group.
    
    Applying functions:
    It takes a function (or a list of functions, or a dictionary mapping column names to functions) as an argument. This function can be a built-in function (e.g., 'sum', 'mean', 'sqrt'), a user-defined function, or a lambda function.
    Broadcasting results:
    If you apply an aggregation function like sum or mean within a groupby().transform() operation, the aggregated value for each group is then assigned to every row belonging to that group in the resulting Series or DataFrame.

"""

In [10]:
# Sample DataFrame
data = {'group': ['A', 'A', 'B', 'B', 'A', 'C', 'C'],
        'value': [10, 15, 20, 25, 12, 30, 35]}
df = pd.DataFrame(data)
df

,group,value
0,A,10
1,A,15
2,B,20
3,B,25
4,A,12
5,C,30
6,C,35


In [11]:
# Calculate the mean of 'value' for each 'group' and add it as a new column
df['group_mean'] = df.groupby('group')['value'].transform(np.mean)
df

,group,value,group_mean
0,A,10,12.333333
1,A,15,12.333333
2,B,20,22.500000
3,B,25,22.500000
4,A,12,12.333333
5,C,30,32.500000
6,C,35,32.500000


In [ ]:
df.groupby('group')['value'].mean()

group
A    12.333333
B    22.500000
C    32.500000
Name: value, dtype: float64

# mock 

In [15]:
employees = pd.DataFrame({
    "employee_id": [101, 102, 103, 104, 105, 106],
    "first_name": ["Alice", "BOB", "Carol", "David", "Emily", "Frank"],
    "last_name": ["Smith", "Jones", "WHITE", "Li", "Chen", "Zhao"],
    "manager_id": [201, 201, 202, 202, None, 201],
    "hire_date": ["2019-03-12", "2020-07-01", "2018-01-09", "2019-11-23", "2021-06-15", "2021-06-15"],
    "department": ["Sales", "Sales", "Marketing", "Marketing", "HR", "Sales"]
})

salaries = pd.DataFrame({
    "employee_id": [101, 101, 102, 103, 104, 105, 106],
    "salary": [75000, 80000, 60000, 70000, 72000, 65000, 62000],
    "effective_date": ["2021-01-01", "2022-01-01", "2021-01-01", "2021-01-01",
                       "2021-01-01", "2021-01-01", "2022-06-01"]
})

In [16]:
employees

,employee_id,first_name,last_name,manager_id,hire_date,department
0,101,Alice,Smith,201.0,2019-03-12,Sales
1,102,BOB,Jones,201.0,2020-07-01,Sales
2,103,Carol,WHITE,202.0,2018-01-09,Marketing
3,104,David,Li,202.0,2019-11-23,Marketing
4,105,Emily,Chen,NaN,2021-06-15,HR
5,106,Frank,Zhao,201.0,2021-06-15,Sales


In [17]:
salaries

,employee_id,salary,effective_date
0,101,75000,2021-01-01
1,101,80000,2022-01-01
2,102,60000,2021-01-01
3,103,70000,2021-01-01
4,104,72000,2021-01-01
5,105,65000,2021-01-01
6,106,62000,2022-06-01


In [27]:
"""
Q1. Data Cleaning

Standardize employee names so that only the first letter of each first/last name is uppercase (rest lowercase).
"""
employees['first_name'] = employees['first_name'].str.capitalize()
employees['last_name'] = employees['last_name'].str.capitalize()
employees

,employee_id,first_name,last_name,manager_id,hire_date,department
0,101,Alice,Smith,201.0,2019-03-12,Sales
1,102,Bob,Jones,201.0,2020-07-01,Sales
2,103,Carol,White,202.0,2018-01-09,Marketing
3,104,David,Li,202.0,2019-11-23,Marketing
4,105,Emily,Chen,NaN,2021-06-15,HR
5,106,Frank,Zhao,201.0,2021-06-15,Sales


In [28]:
"""
Q2. Aggregation

For each manager, find:

earliest hire date of their reports

latest hire date of their reports

number of unique employees reporting to them
"""
employees.groupby('manager_id')['hire_date'].agg(
  earliest_hire='min',
  latest_hire='max'
).reset_index()

,manager_id,earliest_hire,latest_hire
0,201.0,2019-03-12,2021-06-15
1,202.0,2018-01-09,2019-11-23


In [37]:
employees['manager_id'].astype('Int64')

0     201
1     201
2     202
3     202
4    <NA>
5     201
Name: manager_id, dtype: Int64

In [ ]:


data = {'Category': ['A', 'B', 'A', 'C', 'B', 'A'],
        'Value': [10, 20, 15, 25, 30, 12]}
df = pd.DataFrame(data)

grouped = df.groupby('Category')

# iterating through groups
for name, group_df in grouped:
    print(f"Group: {name}")
    print(group_df)
    print("-" * 20)

Group: A
  Category  Value
0        A     10
2        A     15
5        A     12
--------------------
Group: B
  Category  Value
1        B     20
4        B     30
--------------------
Group: C
  Category  Value
3        C     25
--------------------


In [56]:
# access a specific group
group_A = grouped.get_group('A')
print("DataFrame for Group 'A':")
print(group_A)

DataFrame for Group 'A':
  Category  Value
0        A     10
2        A     15
5        A     12


In [57]:
# apply a function to each group

# Example: Calculate the mean of 'Value' for each group
mean_values = grouped['Value'].mean()
print("Mean values per category:")
print(mean_values)

# Example: Apply a custom function to each group
def custom_transform(group):
    group['Value_doubled'] = group['Value'] * 2
    return group

transformed_df = grouped.apply(custom_transform)
print("\nDataFrame with 'Value_doubled' for each group:")
print(transformed_df)

Mean values per category:
Category
A    12.333333
B    25.000000
C    25.000000
Name: Value, dtype: float64

DataFrame with 'Value_doubled' for each group:
           Category  Value  Value_doubled
Category                                 
A        0        A     10             20
         2        A     15             30
         5        A     12             24
B        1        B     20             40
         4        B     30             60
C        3        C     25             50


In [ ]:
"""
176 second highest salary
"""
def second_highest_salary(employee: pd.DataFrame) -> pd.DataFrame:
    # drop duplicates to avoid counting duplicates as separate ranks
    unique_salaries = employee['salary'].drop_duplicates()

    # sort in descending order to get the second highest 
    second = unique_salaries.nlargest(2).iloc[-1] if len(unique_salaries)>=2 else None

    return pd.DataFrame({'SecondHighestSalary': [second]})

In [3]:
df = pd.DataFrame(
  {'id': [1,2,3,4],
  'salary': [100, 200, 200, 300]}
)
df

,id,salary
0,1,100
1,2,200
2,3,200
3,4,300


In [8]:
df2 = df['salary'].drop_duplicates()
df2.dtypes

dtype('int64')